In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_ca.csv") #- original geocoded file
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head() 

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,CA,...,Unanswered,NaN,395264.11,9/10/21,23.0,Construction,SANTA ANA DISTRICT OFFICE,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550
1,35261,35261,1048448901,4/24/21,912,PPP,HUMANGOOD NORCAL,6120 Stoneridge Mall Rd Ste 300,Pleasanton,CA,...,Non-Veteran,NaN,NaN,NaN,62.0,Health Care and Social Assistance,SAN FRANCISCO DISTRICT OFFICE,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918
2,35262,35262,1122827105,4/9/20,914,PPP,ITO INCORPORATED,431 ISIS AVE,INGLEWOOD,CA,...,Non-Veteran,NaN,10112876.71,6/11/21,72.0,Accomodation and Food Services,LOS ANGELES DISTRICT OFFICE,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870
3,35263,35263,1187837109,4/10/20,920,PPP,"HAVEN HEALTH GROUP, LLC","31752 COAST HWY, STE 300",LAGUNA BEACH,CA,...,Non-Veteran,NaN,10116388.89,6/11/21,62.0,Health Care and Social Assistance,SANTA ANA DISTRICT OFFICE,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448
4,35264,35264,1321898800,4/10/21,912,PPP,NORTH EAST MEDICAL SERVICES,2171 Junipero Serra Blvd,Daly City,CA,...,Unanswered,Y,NaN,NaN,62.0,Health Care and Social Assistance,SAN FRANCISCO DISTRICT OFFICE,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #0

,LoanNumber,full_add,Lat,Long


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head()

,LoanNumber,full_add,Lat,Long
0,6223567700,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550
1,1048448901,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918
2,1122827105,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870
3,1187837109,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448
4,1321898800,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [6]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/CA_fips_scraped.csv") 
#fips_check_scraped = fips_check_scraped.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis=1)
fips_check_scraped.head() 

,Unnamed: 0,Unnamed: 0.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,CA,...,N,395264.11,9/10/21,23.0,Construction,SANTA ANA DISTRICT OFFICE,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,6.071005e+13
1,1,35261,1048448901,4/24/21,912,PPP,HUMANGOOD NORCAL,6120 Stoneridge Mall Rd Ste 300,Pleasanton,CA,...,N,NaN,NaN,62.0,Health Care and Social Assistance,SAN FRANCISCO DISTRICT OFFICE,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918,6.001451e+13
2,2,35262,1122827105,4/9/20,914,PPP,ITO INCORPORATED,431 ISIS AVE,INGLEWOOD,CA,...,N,10112876.71,6/11/21,72.0,Accomodation and Food Services,LOS ANGELES DISTRICT OFFICE,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870,6.037601e+13
3,3,35263,1187837109,4/10/20,920,PPP,"HAVEN HEALTH GROUP, LLC","31752 COAST HWY, STE 300",LAGUNA BEACH,CA,...,N,10116388.89,6/11/21,62.0,Health Care and Social Assistance,SANTA ANA DISTRICT OFFICE,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448,6.059042e+13
4,4,35264,1321898800,4/10/21,912,PPP,NORTH EAST MEDICAL SERVICES,2171 Junipero Serra Blvd,Daly City,CA,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,SAN FRANCISCO DISTRICT OFFICE,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386,6.081601e+13


In [7]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [8]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [9]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,full_add,Lat,Long,FIPS_z
0,6223567700,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,6.071005e+13
1,1048448901,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918,6.001451e+13
2,1122827105,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870,6.037601e+13
3,1187837109,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448,6.059042e+13
4,1321898800,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386,6.081601e+13


In [10]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,full_add,Lat,Long,FIPS_z
16343,4861237004,"2865 South Jones Blvd, SHERMAN OAKS, CA",36.137302,-115.225643,NaN
46854,5428878008,"122 ORCHARD DR, CAMPBELL, CA",40.080060,-80.889337,NaN
57030,6471047010,"47110 WASHINGTON ST STE 201, LA QUINTA, CA",46.572476,-122.905019,NaN
58542,7948857804,"19912 Stateline Road, Tulelake, CA",41.997363,-121.435929,NaN
61746,6089678409,"19912 Stateline Rd, Tulelake, CA",41.997363,-121.435929,NaN
...,...,...,...,...,...
130617,9980578504,"16633 Ventura Blvd Ste 913, Encino, CA",34.158291,-118.495278,NaN
130618,9985047708,"10749 ROSELLE ST, SAN DIEGO, CA",32.899195,-117.222392,NaN
130619,9985407709,"11180 WARNER AVENUE, FOUNTAIN VALLEY, CA",33.715426,-117.933275,NaN
130620,9989087303,"946 Santa Rosa Ave, SANTA ROSA, CA",38.429187,-122.711481,NaN


<h4>FIPS Script

In [11]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [12]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('06')] #CA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_mac64.zip
Driver has been saved in cache [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/98.0.4758.102]


coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [13]:
geo_id_list =geo_id 

Length Check

In [14]:
print(len(geo_id_list))

28093


Matching FIPS to DataFrame

In [15]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:2182] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null.iloc[:28093]

In [16]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
16343,4861237004,"2865 South Jones Blvd, SHERMAN OAKS, CA",36.137302,-115.225643,[]
46854,5428878008,"122 ORCHARD DR, CAMPBELL, CA",40.080060,-80.889337,[]
57030,6471047010,"47110 WASHINGTON ST STE 201, LA QUINTA, CA",46.572476,-122.905019,[]
58542,7948857804,"19912 Stateline Road, Tulelake, CA",41.997363,-121.435929,[]
61746,6089678409,"19912 Stateline Rd, Tulelake, CA",41.997363,-121.435929,[]
...,...,...,...,...,...
130617,9980578504,"16633 Ventura Blvd Ste 913, Encino, CA",34.158291,-118.495278,[060371397012008]
130618,9985047708,"10749 ROSELLE ST, SAN DIEGO, CA",32.899195,-117.222392,[060730083391037]
130619,9985407709,"11180 WARNER AVENUE, FOUNTAIN VALLEY, CA",33.715426,-117.933275,[060590992294002]
130620,9989087303,"946 Santa Rosa Ave, SANTA ROSA, CA",38.429187,-122.711481,[060971514061015]


Remove brackets

In [17]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
16343,4861237004,"2865 South Jones Blvd, SHERMAN OAKS, CA",36.137302,-115.225643,NaN
46854,5428878008,"122 ORCHARD DR, CAMPBELL, CA",40.080060,-80.889337,NaN
57030,6471047010,"47110 WASHINGTON ST STE 201, LA QUINTA, CA",46.572476,-122.905019,NaN
58542,7948857804,"19912 Stateline Road, Tulelake, CA",41.997363,-121.435929,NaN
61746,6089678409,"19912 Stateline Rd, Tulelake, CA",41.997363,-121.435929,NaN
...,...,...,...,...,...
130617,9980578504,"16633 Ventura Blvd Ste 913, Encino, CA",34.158291,-118.495278,060371397012008
130618,9985047708,"10749 ROSELLE ST, SAN DIEGO, CA",32.899195,-117.222392,060730083391037
130619,9985407709,"11180 WARNER AVENUE, FOUNTAIN VALLEY, CA",33.715426,-117.933275,060590992294002
130620,9989087303,"946 Santa Rosa Ave, SANTA ROSA, CA",38.429187,-122.711481,060971514061015


In [18]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #17

,LoanNumber,full_add,Lat,Long,FIPS_z
16343,4861237004,"2865 South Jones Blvd, SHERMAN OAKS, CA",36.137302,-115.225643,NaN
46854,5428878008,"122 ORCHARD DR, CAMPBELL, CA",40.080060,-80.889337,NaN
57030,6471047010,"47110 WASHINGTON ST STE 201, LA QUINTA, CA",46.572476,-122.905019,NaN
58542,7948857804,"19912 Stateline Road, Tulelake, CA",41.997363,-121.435929,NaN
61746,6089678409,"19912 Stateline Rd, Tulelake, CA",41.997363,-121.435929,NaN
77162,1093197302,"5959 Commerce Blvd, Isle-Aux-Morts, CA",47.587461,-58.974439,NaN
89923,7414237300,"1310 Millet St, Anaheim, CA",36.634356,-87.301086,NaN
99793,5027317007,"2865 South Jones, Blvd., SHERMAN OAKS, CA",36.137302,-115.225643,NaN
113702,9361287200,"8816 FOOTHILL BLVD #103-131, RANCHO CUCAMONGA, CA",29.875339,-152.279104,NaN
114496,4126958607,"8816 Foothill Blvd Ste 103-393, Rancho Cucamon...",29.875339,-152.279104,NaN


Parse for Merge

In [19]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
16343,4861237004,NaN
46854,5428878008,NaN
57030,6471047010,NaN
58542,7948857804,NaN
61746,6089678409,NaN
...,...,...
130617,9980578504,060371397012008
130618,9985047708,060730083391037
130619,9985407709,060590992294002
130620,9989087303,060971514061015


Read in base data

In [20]:
#if from geocode file
#fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_ca.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
#fips_check

#if from FIPS file
fips_check = pd.read_csv("data/state_data/geo/geo_fips/CA_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,0,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,...,N,395264.11,9/10/21,23.0,Construction,SANTA ANA DISTRICT OFFICE,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,6.071005e+13
1,1,1,1,35261,1048448901,4/24/21,912,PPP,HUMANGOOD NORCAL,6120 Stoneridge Mall Rd Ste 300,...,N,NaN,NaN,62.0,Health Care and Social Assistance,SAN FRANCISCO DISTRICT OFFICE,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918,6.001451e+13
2,2,2,2,35262,1122827105,4/9/20,914,PPP,ITO INCORPORATED,431 ISIS AVE,...,N,10112876.71,6/11/21,72.0,Accomodation and Food Services,LOS ANGELES DISTRICT OFFICE,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870,6.037601e+13
3,3,3,3,35263,1187837109,4/10/20,920,PPP,"HAVEN HEALTH GROUP, LLC","31752 COAST HWY, STE 300",...,N,10116388.89,6/11/21,62.0,Health Care and Social Assistance,SANTA ANA DISTRICT OFFICE,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448,6.059042e+13
4,4,4,4,35264,1321898800,4/10/21,912,PPP,NORTH EAST MEDICAL SERVICES,2171 Junipero Serra Blvd,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,SAN FRANCISCO DISTRICT OFFICE,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386,6.081601e+13


Merge with MainFrame

In [21]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,0,0,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,...,395264.11,9/10/21,23.0,Construction,SANTA ANA DISTRICT OFFICE,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,6.071005e+13,NaN
1,1,1,1,35261,1048448901,4/24/21,912,PPP,HUMANGOOD NORCAL,6120 Stoneridge Mall Rd Ste 300,...,NaN,NaN,62.0,Health Care and Social Assistance,SAN FRANCISCO DISTRICT OFFICE,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918,6.001451e+13,NaN
2,2,2,2,35262,1122827105,4/9/20,914,PPP,ITO INCORPORATED,431 ISIS AVE,...,10112876.71,6/11/21,72.0,Accomodation and Food Services,LOS ANGELES DISTRICT OFFICE,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870,6.037601e+13,NaN
3,3,3,3,35263,1187837109,4/10/20,920,PPP,"HAVEN HEALTH GROUP, LLC","31752 COAST HWY, STE 300",...,10116388.89,6/11/21,62.0,Health Care and Social Assistance,SANTA ANA DISTRICT OFFICE,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448,6.059042e+13,NaN
4,4,4,4,35264,1321898800,4/10/21,912,PPP,NORTH EAST MEDICAL SERVICES,2171 Junipero Serra Blvd,...,NaN,NaN,62.0,Health Care and Social Assistance,SAN FRANCISCO DISTRICT OFFICE,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386,6.081601e+13,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130617,130617,130617,130617,165877,9980578504,3/12/21,914,PPS,FOUR SQUARE CORPORATION,16633 Ventura Blvd Ste 913,...,150793.15,9/28/21,71.0,"Arts, Entertainment, and Recreation",LOS ANGELES DISTRICT OFFICE,"16633 Ventura Blvd Ste 913, Encino, CA",34.158291,-118.495278,NaN,060371397012008
130618,130618,130618,130618,165878,9985047708,5/3/20,954,PPP,GW CRUSBERG INC,10749 ROSELLE ST,...,NaN,NaN,23.0,Construction,SAN DIEGO DISTRICT OFFICE,"10749 ROSELLE ST, SAN DIEGO, CA",32.899195,-117.222392,NaN,060730083391037
130619,130619,130619,130619,165879,9985407709,5/1/20,920,PPP,LALITA PANDIT MD. INC.,11180 WARNER AVENUE,...,151491.78,5/6/21,62.0,Health Care and Social Assistance,SANTA ANA DISTRICT OFFICE,"11180 WARNER AVENUE, FOUNTAIN VALLEY, CA",33.715426,-117.933275,NaN,060590992294002
130620,130620,130620,130620,165880,9989087303,5/3/20,912,PPP,"SANTA ROSA SEAFOOD RETAIL, INC.",946 Santa Rosa Ave,...,151318.92,3/25/21,44.0,Retail Trade,SAN FRANCISCO DISTRICT OFFICE,"946 Santa Rosa Ave, SANTA ROSA, CA",38.429187,-122.711481,NaN,060971514061015


<b>Fill FIPS Columns - if FIPS file

In [22]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,0,0,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,...,9/10/21,23.0,Construction,SANTA ANA DISTRICT OFFICE,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,6.071005e+13,NaN,6.071e+13
1,1,1,1,35261,1048448901,4/24/21,912,PPP,HUMANGOOD NORCAL,6120 Stoneridge Mall Rd Ste 300,...,NaN,62.0,Health Care and Social Assistance,SAN FRANCISCO DISTRICT OFFICE,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918,6.001451e+13,NaN,6.00145e+13
2,2,2,2,35262,1122827105,4/9/20,914,PPP,ITO INCORPORATED,431 ISIS AVE,...,6/11/21,72.0,Accomodation and Food Services,LOS ANGELES DISTRICT OFFICE,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870,6.037601e+13,NaN,6.0376e+13
3,3,3,3,35263,1187837109,4/10/20,920,PPP,"HAVEN HEALTH GROUP, LLC","31752 COAST HWY, STE 300",...,6/11/21,62.0,Health Care and Social Assistance,SANTA ANA DISTRICT OFFICE,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448,6.059042e+13,NaN,6.05904e+13
4,4,4,4,35264,1321898800,4/10/21,912,PPP,NORTH EAST MEDICAL SERVICES,2171 Junipero Serra Blvd,...,NaN,62.0,Health Care and Social Assistance,SAN FRANCISCO DISTRICT OFFICE,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386,6.081601e+13,NaN,6.0816e+13


Drop Extra FIPS Columns

In [23]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1","Unnamed: 0.1.1", "Unnamed: 0.1.1.1", "Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,0,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,...,N,395264.11,9/10/21,23.0,Construction,SANTA ANA DISTRICT OFFICE,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,6.071e+13
1,1,1,1,35261,1048448901,4/24/21,912,PPP,HUMANGOOD NORCAL,6120 Stoneridge Mall Rd Ste 300,...,N,NaN,NaN,62.0,Health Care and Social Assistance,SAN FRANCISCO DISTRICT OFFICE,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918,6.00145e+13
2,2,2,2,35262,1122827105,4/9/20,914,PPP,ITO INCORPORATED,431 ISIS AVE,...,N,10112876.71,6/11/21,72.0,Accomodation and Food Services,LOS ANGELES DISTRICT OFFICE,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870,6.0376e+13
3,3,3,3,35263,1187837109,4/10/20,920,PPP,"HAVEN HEALTH GROUP, LLC","31752 COAST HWY, STE 300",...,N,10116388.89,6/11/21,62.0,Health Care and Social Assistance,SANTA ANA DISTRICT OFFICE,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448,6.05904e+13
4,4,4,4,35264,1321898800,4/10/21,912,PPP,NORTH EAST MEDICAL SERVICES,2171 Junipero Serra Blvd,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,SAN FRANCISCO DISTRICT OFFICE,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386,6.0816e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130617,130617,130617,130617,165877,9980578504,3/12/21,914,PPS,FOUR SQUARE CORPORATION,16633 Ventura Blvd Ste 913,...,N,150793.15,9/28/21,71.0,"Arts, Entertainment, and Recreation",LOS ANGELES DISTRICT OFFICE,"16633 Ventura Blvd Ste 913, Encino, CA",34.158291,-118.495278,060371397012008
130618,130618,130618,130618,165878,9985047708,5/3/20,954,PPP,GW CRUSBERG INC,10749 ROSELLE ST,...,N,NaN,NaN,23.0,Construction,SAN DIEGO DISTRICT OFFICE,"10749 ROSELLE ST, SAN DIEGO, CA",32.899195,-117.222392,060730083391037
130619,130619,130619,130619,165879,9985407709,5/1/20,920,PPP,LALITA PANDIT MD. INC.,11180 WARNER AVENUE,...,N,151491.78,5/6/21,62.0,Health Care and Social Assistance,SANTA ANA DISTRICT OFFICE,"11180 WARNER AVENUE, FOUNTAIN VALLEY, CA",33.715426,-117.933275,060590992294002
130620,130620,130620,130620,165880,9989087303,5/3/20,912,PPP,"SANTA ROSA SEAFOOD RETAIL, INC.",946 Santa Rosa Ave,...,N,151318.92,3/25/21,44.0,Retail Trade,SAN FRANCISCO DISTRICT OFFICE,"946 Santa Rosa Ave, SANTA ROSA, CA",38.429187,-122.711481,060971514061015


In [25]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #17/130622

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
16343,16343,16343,16343,51603,4861237004,4/4/20,914,PPP,"DYNAMIC HOME HEALTH CARE, LLC",2865 South Jones Blvd,...,N,945176.85,6/11/21,62.0,Health Care and Social Assistance,LOS ANGELES DISTRICT OFFICE,"2865 South Jones Blvd, SHERMAN OAKS, CA",36.137302,-115.225643,NaN
46854,46854,46854,46854,82114,5428878008,6/28/20,912,PPP,"GRIDLEY CONSTRUCTION, INC",122 ORCHARD DR,...,N,408935.66,10/15/21,23.0,Construction,SAN FRANCISCO DISTRICT OFFICE,"122 ORCHARD DR, CAMPBELL, CA",40.080060,-80.889337,NaN
57030,57030,57030,57030,92290,6471047010,4/7/20,920,PPP,VALLEY PIPELINE SERVICES INC,47110 WASHINGTON ST STE 201,...,N,341537.10,3/5/21,23.0,Construction,SANTA ANA DISTRICT OFFICE,"47110 WASHINGTON ST STE 201, LA QUINTA, CA",46.572476,-122.905019,NaN
58542,58542,58542,58542,93802,7948857804,6/4/20,931,PPP,"CAL-ORE PRODUCE, INC.",19912 Stateline Road,...,N,NaN,NaN,11.0,"Agriculture, Forestry, Fishing and Hunting",SACRAMENTO DISTRICT OFFICE,"19912 Stateline Road, Tulelake, CA",41.997363,-121.435929,NaN
61746,61746,61746,61746,97006,6089678409,2/10/21,931,PPS,CAL ORE PRODUCE,19912 Stateline Rd,...,N,NaN,NaN,11.0,"Agriculture, Forestry, Fishing and Hunting",SACRAMENTO DISTRICT OFFICE,"19912 Stateline Rd, Tulelake, CA",41.997363,-121.435929,NaN
77162,77162,77162,77162,112422,1093197302,4/28/20,912,PPP,"HP INVESTMENTS, INC",5959 Commerce Blvd,...,N,NaN,NaN,53.0,Real Estate Rental and Leasing,SAN FRANCISCO DISTRICT OFFICE,"5959 Commerce Blvd, Isle-Aux-Morts, CA",47.587461,-58.974439,NaN
89923,89923,89923,89923,125183,7414237300,4/30/20,920,PPP,"CASE & SONS CONSTRUCTION, INC.",1310 Millet St,...,N,222219.39,5/11/21,23.0,Construction,SANTA ANA DISTRICT OFFICE,"1310 Millet St, Anaheim, CA",36.634356,-87.301086,NaN
99793,99793,99793,99793,135053,5027317007,4/4/20,914,PPP,A SIMPLE SOLUTION LLC,"2865 South Jones, Blvd.",...,N,201031.53,3/11/21,62.0,Health Care and Social Assistance,LOS ANGELES DISTRICT OFFICE,"2865 South Jones, Blvd., SHERMAN OAKS, CA",36.137302,-115.225643,NaN
113702,113702,113702,113702,148962,9361287200,4/28/20,920,PPP,"SPECIALTY PIPING SERVICES, INC.",8816 FOOTHILL BLVD #103-131,...,N,NaN,NaN,81.0,Other Services (except Public Administration),SANTA ANA DISTRICT OFFICE,"8816 FOOTHILL BLVD #103-131, RANCHO CUCAMONGA, CA",29.875339,-152.279104,NaN
114496,114496,114496,114496,149756,4126958607,3/18/21,920,PPS,THE ELEVATOR COMPANY INC. A CORPORATION,8816 Foothill Blvd Ste 103-393,...,N,173233.13,8/25/21,23.0,Construction,SANTA ANA DISTRICT OFFICE,"8816 Foothill Blvd Ste 103-393, Rancho Cucamon...",29.875339,-152.279104,NaN


Convert to File

In [26]:
#fips_CA = fips_merge
fips_CA = fips_merge_drop

In [27]:
#pd.set_option('display.max_columns', None)
#fips_CA["NonProfit"] = fips_CA["NonProfit"].fillna('N')
#fips_CA

In [28]:
fips_CA.to_csv("data/state_data/geo/geo_fips/CA_fips_scraped.csv") 

Review & Compare

In [29]:
geo_CA_fips = pd.read_csv("data/state_data/geo/geo_fips/CA_fips_scraped.csv") 
geo_CA_fips.head() #130622

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,0,0,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,...,N,395264.11,9/10/21,23.0,Construction,SANTA ANA DISTRICT OFFICE,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,6.071005e+13
1,1,1,1,1,35261,1048448901,4/24/21,912,PPP,HUMANGOOD NORCAL,...,N,NaN,NaN,62.0,Health Care and Social Assistance,SAN FRANCISCO DISTRICT OFFICE,"6120 Stoneridge Mall Rd Ste 300, Pleasanton, CA",37.698909,-121.924918,6.001451e+13
2,2,2,2,2,35262,1122827105,4/9/20,914,PPP,ITO INCORPORATED,...,N,10112876.71,6/11/21,72.0,Accomodation and Food Services,LOS ANGELES DISTRICT OFFICE,"431 ISIS AVE, INGLEWOOD, CA",33.957520,-118.375870,6.037601e+13
3,3,3,3,3,35263,1187837109,4/10/20,920,PPP,"HAVEN HEALTH GROUP, LLC",...,N,10116388.89,6/11/21,62.0,Health Care and Social Assistance,SANTA ANA DISTRICT OFFICE,"31752 COAST HWY, STE 300, LAGUNA BEACH, CA",33.501729,-117.743448,6.059042e+13
4,4,4,4,4,35264,1321898800,4/10/21,912,PPP,NORTH EAST MEDICAL SERVICES,...,Y,NaN,NaN,62.0,Health Care and Social Assistance,SAN FRANCISCO DISTRICT OFFICE,"2171 Junipero Serra Blvd, Daly City, CA",37.698365,-122.470386,6.081601e+13


In [30]:
##### fill in missing Lat, Long, FIPS
geo_CA_fips.loc[geo_CA_fips["FIPS_z"].isnull()] #328093

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
16343,16343,16343,16343,16343,51603,4861237004,4/4/20,914,PPP,"DYNAMIC HOME HEALTH CARE, LLC",...,N,945176.85,6/11/21,62.0,Health Care and Social Assistance,LOS ANGELES DISTRICT OFFICE,"2865 South Jones Blvd, SHERMAN OAKS, CA",36.137302,-115.225643,NaN
46854,46854,46854,46854,46854,82114,5428878008,6/28/20,912,PPP,"GRIDLEY CONSTRUCTION, INC",...,N,408935.66,10/15/21,23.0,Construction,SAN FRANCISCO DISTRICT OFFICE,"122 ORCHARD DR, CAMPBELL, CA",40.080060,-80.889337,NaN
57030,57030,57030,57030,57030,92290,6471047010,4/7/20,920,PPP,VALLEY PIPELINE SERVICES INC,...,N,341537.10,3/5/21,23.0,Construction,SANTA ANA DISTRICT OFFICE,"47110 WASHINGTON ST STE 201, LA QUINTA, CA",46.572476,-122.905019,NaN
58542,58542,58542,58542,58542,93802,7948857804,6/4/20,931,PPP,"CAL-ORE PRODUCE, INC.",...,N,NaN,NaN,11.0,"Agriculture, Forestry, Fishing and Hunting",SACRAMENTO DISTRICT OFFICE,"19912 Stateline Road, Tulelake, CA",41.997363,-121.435929,NaN
61746,61746,61746,61746,61746,97006,6089678409,2/10/21,931,PPS,CAL ORE PRODUCE,...,N,NaN,NaN,11.0,"Agriculture, Forestry, Fishing and Hunting",SACRAMENTO DISTRICT OFFICE,"19912 Stateline Rd, Tulelake, CA",41.997363,-121.435929,NaN
77162,77162,77162,77162,77162,112422,1093197302,4/28/20,912,PPP,"HP INVESTMENTS, INC",...,N,NaN,NaN,53.0,Real Estate Rental and Leasing,SAN FRANCISCO DISTRICT OFFICE,"5959 Commerce Blvd, Isle-Aux-Morts, CA",47.587461,-58.974439,NaN
89923,89923,89923,89923,89923,125183,7414237300,4/30/20,920,PPP,"CASE & SONS CONSTRUCTION, INC.",...,N,222219.39,5/11/21,23.0,Construction,SANTA ANA DISTRICT OFFICE,"1310 Millet St, Anaheim, CA",36.634356,-87.301086,NaN
99793,99793,99793,99793,99793,135053,5027317007,4/4/20,914,PPP,A SIMPLE SOLUTION LLC,...,N,201031.53,3/11/21,62.0,Health Care and Social Assistance,LOS ANGELES DISTRICT OFFICE,"2865 South Jones, Blvd., SHERMAN OAKS, CA",36.137302,-115.225643,NaN
113702,113702,113702,113702,113702,148962,9361287200,4/28/20,920,PPP,"SPECIALTY PIPING SERVICES, INC.",...,N,NaN,NaN,81.0,Other Services (except Public Administration),SANTA ANA DISTRICT OFFICE,"8816 FOOTHILL BLVD #103-131, RANCHO CUCAMONGA, CA",29.875339,-152.279104,NaN
114496,114496,114496,114496,114496,149756,4126958607,3/18/21,920,PPS,THE ELEVATOR COMPANY INC. A CORPORATION,...,N,173233.13,8/25/21,23.0,Construction,SANTA ANA DISTRICT OFFICE,"8816 Foothill Blvd Ste 103-393, Rancho Cucamon...",29.875339,-152.279104,NaN
